<center><h2>N - Queens Problem</h2></center><br>

Place N number of Queens on a Chessboard with size N * N in a way that not a single Queen is attacking another.
This problem is considered NP-Complete because as the value for N increases it becomes exponentially more complicated to solve using brute force but if a solution is provided, the validity of that solution can be confirmed in polynomial time.

The code below is using a heuristic approach to solve this in a hope that even when assigning large values for N, the problem can be solved fairly easily.

This Genetic Algorithm(GA) allows us to try multiple solutions, evaluate them and by merging the best solutions quickly come to an even better solution.
Each individual represents a possible solution to the problem, the "fitness" of the solution is calculated based on how many Queens are attaching each other. Individuals with lower fitness are better solutions.
The GA runs for multiple generations and from each generation individuals are picked for reproduction. They can be picked randomly or based on their fitness. Once two parents are picked, they create a child or children using some kind of crossover function. This new individual created might use some form of mutation to introduce a random change into its solution hopefully in a way that is beneficial for solving the problem. When the new generation has been created, each individual's fitness is evaluated and the process starts again until we find our solution or we hit the limit for number of generations in which case we have failed to find a solution to the problem

In [1]:
import enum
import numpy as np
import random
import sys

random.seed(12345)

In [2]:
class Individual():
    def __init__(self, _geneSize, _initGenes):
        self.geneSize = _geneSize
        self.genes = [-1] * self.geneSize
        self.fitness = sys.maxsize

        if _initGenes:
            self.InitialiseGenesRandomly()
    
    def InitialiseGenesRandomly(self):
        for i in range(0, self.geneSize):
            self.genes[i] = random.randint(0, self.geneSize-1)
        self.calculateFitness()
        
    def getFitness(self):
        return self.fitness
    
    # Calculate fitness based on the number of queens attacking each other using the current solution
    def calculateFitness(self):
        # is intersecting in row?
        totalCost = 0
        for i in range(0, self.geneSize):
            ind = self.genes[i]
            #print('Checking {} with val: {}'.format(i, ind))
            for r in range(1, self.geneSize-i):
                #print('R is: {}'.format(r))
                if self.genes[i+r] == ind:
                    totalCost += 1
                    #print('Adding cost because of cross')
                # is intersecting diagonally?
                if self.genes[r+i] == ind+r or self.genes[r+i] == ind-r:
                    totalCost += 1
                    #print('Adding cost because of diagonal')
        self.fitness = totalCost
        
    def copy(self):
        newInd = Individual(self.geneSize, False)
        for i in range(0, self.geneSize):
            newInd.genes[i] = self.genes[i]
        newInd.calculateFitness()
        return newInd

In [3]:
# CROSSOVER
def UniformOrderCrossover(_parentOne, _parentTwo, _geneSize):
    newGenes = [-1] * _geneSize
    for i in range(0, _geneSize):
        rand = random.random()
        if rand < 0.5:
            newGenes[i] = _parentOne[i]
        else:
            newGenes[i] = _parentTwo[i]

    return newGenes

In [4]:
# MUTATION
# reciprocal exchange mutation:
# randomly pick 2 genes along the chromosome and swap them
def reciprocalExchangeMutation(_genes, _mutationRate, _geneSize):
    if random.random() > _mutationRate:
        return _genes

    randIndexOne = -1
    randIndexTwo = -1
    while randIndexOne == randIndexTwo:
        randIndexOne = random.randint(0, _geneSize-1)
        randIndexTwo = random.randint(0, _geneSize-1)
    
    temp = _genes[randIndexOne]
    _genes[randIndexOne] = _genes[randIndexTwo]
    _genes[randIndexTwo] = temp
    
    return _genes

In [5]:
class GA():
    def __init__(self, _popSize, _numIters, _geneSize, _mutationRate):
        self.numIters   = _numIters
        self.popSize    = _popSize
        self.population = [None] * self.popSize
        self.matingPool = []
        
        self.geneSize      = _geneSize
        self.mutationRatio = _mutationRate
        
        self.bestIndividual = None
        
        self.InitialisePopulation()
        self.CalculateBestFitness()
        self.Run()
    
    def InitialisePopulation(self):
        for i in range(0, self.popSize):
            self.population[i] = Individual(self.geneSize, True)
    
    def Run(self):
        for i in range(self.numIters):
            print('Running Iteration: {}'.format(i))
            self.Step()
            if self.bestIndividual.getFitness() == 0:
                print('Early finish. Found best solution in iteration: {}'.format(i))
                self.ShowBestSolution()
                return
        self.ShowBestSolution()
    
    def CalculateBestFitness(self):
        best = self.population[0]
        for i in range(1, self.popSize):
            if self.population[i].getFitness() < best.getFitness():
                best = self.population[i].copy()
                
        if self.bestIndividual == None or best.getFitness() < self.bestIndividual.getFitness():
            print('Updating best fitness to: {}'.format(best.getFitness()))
            self.bestIndividual = best.copy()
    
    def Step(self):
        # Step 1: Create new mating pool
        self.CreateMatingPool()
        
        for i in range(0, self.popSize):
            child = Individual(self.geneSize, False)
            # Step 2: Selection of Parents
            parents = self.RandomSelection()
            # Step 3: Crossover
            childGenes = UniformOrderCrossover(parents[0].genes, parents[1].genes, self.geneSize)
            # Step 4: Mutation
            childGenes = reciprocalExchangeMutation(childGenes, self.mutationRatio, self.geneSize)
            # Step 5: Replace old population
            child.genes = childGenes
            child.calculateFitness()
            self.population[i] = child
        
        self.CalculateBestFitness()
    
    def RandomSelection(self):
        parentOne = self.matingPool[random.randint(0, self.popSize-1)]
        parentTwo = self.matingPool[random.randint(0, self.popSize-1)]
        return [parentOne, parentTwo]
    
    # using Stochastic Universal Sampling for creating the mating pool
    def CreateMatingPool(self):
        maxFitness = 0.0
        for i in range(0, self.popSize):
            fitness = self.population[i].getFitness()
            if maxFitness < fitness:
                maxFitness = fitness
        
        fitnessProbs = []
        totalRelativeFitness = 0.0
        for i in range(0, self.popSize):
            fitnessRelative = maxFitness - self.population[i].getFitness() + 1
            totalRelativeFitness += fitnessRelative
            fitnessProbs.append(fitnessRelative)
        
        for i in range(0, self.popSize):
            relative = fitnessProbs[i]
            fitnessProbs[i] = relative / totalRelativeFitness
            
        susStepSize = 1/self.popSize
        susStartPoint = random.uniform(0.0, susStepSize)
        current = susStartPoint
        index = 0
        previousFitness = 0.0
        
        self.matingPool = []
        while current < 1.0:
            indivFitness = previousFitness + fitnessProbs[index]
            if indivFitness >= current:
                # The same indidividual could get picked multiple times
                while indivFitness >= current and current < 1.0:
                    self.matingPool.append(self.population[index].copy())
                    current += susStepSize

            index += 1
            previousFitness = indivFitness

        if len(self.matingPool) != self.popSize:
            print('ERROR!!! stochasticUniversalSampling has created a mating pool with size: {}'.format(len(self.matingPool)))

    
    
    def ShowBestSolution(self):
        lines = [""] * self.geneSize
        for row in range(0, self.geneSize):
            for col in range(0, self.geneSize):
                if self.bestIndividual.genes[row] == col:
                    lines[col] += 'X '
                else:
                    lines[col] += '0 '
                    
        print("Best Result with cost:{} is: {}".format(self.bestIndividual.getFitness() ,self.bestIndividual.genes))
        for row in range(0, self.geneSize):
            print(lines[row])

In [6]:
NQueen = 6

populationSize = 100
numIterations  = 100
mutationRate   = 0.01

ga = GA(populationSize, numIterations, NQueen, mutationRate)

Updating best fitness to: 2
Running Iteration: 0
Running Iteration: 1
Updating best fitness to: 1
Running Iteration: 2
Running Iteration: 3
Running Iteration: 4
Running Iteration: 5
Running Iteration: 6
Running Iteration: 7
Running Iteration: 8
Running Iteration: 9
Running Iteration: 10
Running Iteration: 11
Running Iteration: 12
Running Iteration: 13
Updating best fitness to: 0
Early finish. Found best solution in iteration: 13
Best Result with cost:0 is: [2, 5, 1, 4, 0, 3]
0 0 0 0 X 0 
0 0 X 0 0 0 
X 0 0 0 0 0 
0 0 0 0 0 X 
0 0 0 X 0 0 
0 X 0 0 0 0 
